In [2]:
from pyspark.sql import SparkSession, Row
from pyspark import SparkContext, SparkConf

# Create SparkSession

In [3]:
session = SparkSession.builder.appName('Ops').getOrCreate()
conf = SparkConf()
sc = session.sparkContext


# Read tsv file into 

In [5]:
path = input("Enter path to input file: ")
#path = "/home/rcdc008/work/cs226/assn3/spark-basics/nasa.tsv"
print("Getting tsv at path", path)
inputFile = sc.textFile(path)

Getting tsv at path /home/rcdc008/work/cs226/assn3/spark-basics/nasa.tsv


# Part A (compute average by response code)

In [73]:
#maps each line's response code to its bytes as a list
responseCodeRDD = inputFile.map(lambda line: (line.split("\t")[5], [float(line.split("\t")[6])]))
#reduceByKey to create a big list for each distinct key
responseCodeListRDD = responseCodeRDD.reduceByKey(lambda a, b: a+b)
#map to get each response code's avg
responseCodeAvgRDD = responseCodeListRDD.map(lambda line: (line[0], sum(line[1])/len(line[1])))

# Part A output

In [93]:
task1 = open("task1.txt", "w")
for entry in responseCodeAvgRDD.take(999999999):
    task1.write("Code " + entry[0] + ", average number of bytes = " + str(entry[1]) + "\n")
    
task1.close()

Code 304 has average number of bytes = 0.0
Code 200 has average number of bytes = 22848.716743119265
Code 302 has average number of bytes = 73.54901960784314
Code 404 has average number of bytes = 0.0


# Part B (pairs of requests satisfying join conditions)

In [29]:
#create rdd where key = (host, url) and value = line
hostAndURLRDD = inputFile.map(lambda line: ((line.split("\t")[0], line.split("\t")[4]), line))
#join that rdd with itself get all pairs of lines with matching host and url (conditions a and b in specs)
partB = hostAndURLRDD.join(hostAndURLRDD)

In [36]:
#filter out lines that are the same (condition d in specs)
partB = partB.filter(lambda x: x[1][0] != x[1][1])
#filter out kv pairs where timestamp are more than 3600 apart (condition c in specs)
partB = partB.filter(lambda x: abs(int(x[1][0].split("\t")[2]) - int(x[1][1].split("\t")[2])) <= 3600)

# Part B output

In [42]:
def cleanOutput(record1: str, record2: str) -> str:
    record1 = "(" + record1.replace("\t", ",")
    while record1[len(record1) - 1] == ',':
        record1 = record1[0:len(record1)-1]
    record1 += ")"
    
    record2 = "(" + record2.replace("\t", ",")
    while record2[len(record2) - 1] == ',':
        record2 = record2[0:len(record2)-1]
    record2 += ")"
    
    return record1 + "\n" + record2
    
task2 = open("task2.txt", "w")
for entry in partB.take(999999999):
    task2.write(entry[1][0] + "\t" + entry[1][1] + "\n")
    
task2.close()

alyssa.prodigy.com	-	804571233	GET	/shuttle/missions/sts-71/sts-71-patch-small.gif	200	12054			alyssa.prodigy.com	-	804571543	GET	/shuttle/missions/sts-71/sts-71-patch-small.gif	200	12054		

alyssa.prodigy.com	-	804571233	GET	/shuttle/missions/sts-71/sts-71-patch-small.gif	200	12054			alyssa.prodigy.com	-	804571602	GET	/shuttle/missions/sts-71/sts-71-patch-small.gif	200	12054		

alyssa.prodigy.com	-	804571233	GET	/shuttle/missions/sts-71/sts-71-patch-small.gif	200	12054			alyssa.prodigy.com	-	804571615	GET	/shuttle/missions/sts-71/sts-71-patch-small.gif	200	12054		

alyssa.prodigy.com	-	804571543	GET	/shuttle/missions/sts-71/sts-71-patch-small.gif	200	12054			alyssa.prodigy.com	-	804571233	GET	/shuttle/missions/sts-71/sts-71-patch-small.gif	200	12054		

alyssa.prodigy.com	-	804571543	GET	/shuttle/missions/sts-71/sts-71-patch-small.gif	200	12054			alyssa.prodigy.com	-	804571602	GET	/shuttle/missions/sts-71/sts-71-patch-small.gif	200	12054		

alyssa.prodigy.com	-	804571543	GET	/shuttle/m